In [1]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
to_run = True
if to_run:
    os.chdir('..')
    to_run = False
print(os.getcwd())

/Users/aninauldum/Documents/Datalogi/REBS/opg1/notebooks
/Users/aninauldum/Documents/Datalogi/REBS/opg1


# Group Details

**Group ID:** Nr. 8

**Group name:** Mandatory Assignments 8 

**Group members that contributed to the assignment (full name):** Phillip Schack Linnemann Lundin & Anina Lyck Uldum 


# Part 1


Consider the following activities extracted from the Dreyers log introduced and examined in the paper The Analysis of a Real Life Declarative Process:

```python
Fill out application
Architect Review
Lawyer Review
Reject
Applicant informed
Change phase to Abort
Change Phase to Payout
First payment
Undo payment 
Change Phase to End Report
Account number changed
Approve changed account 
Execute abandon
Change phase to Abandon
```


## Task 1


Using these activities, model a simple workflow of an application process as a Petri net. It does not need to follow exactly the process as described in the paper and you may add additional details yourself, but you should ensure that your process captures at least the following rules:

1. Fill out application should always be the first event of the case. 
2. Lawyer Review and Architect Review can not occur unless Fill out application happened before.
3. Lawyer Review and Architect Review should never occur together.
4. Reject should always eventually be followed by Applicant informed and Change phase to Abort.
5. First payment should only occur once, unless Undo payment is executed afterwards, in which case it may be repeated once more.
6. If Account number changed happens, then afterwards Approve changed account needs to be executed and we can no longer execute First payment until the approval has happened.
7. Change Phase to Payout should always eventually be followed by First payment.
8. After Change Phase to Payout has happened, Change Phase to End Report should not happen before we do First Payment. If Change Phase to Payout did not happen then Change Phase to End Report is not restricted by First payment.

Create a copy of your Petrinet with one additional rule:

9. Execute Abandon may happen at any time, after it is executed only Change phase to Abandon may happen.

In [3]:
import pandas as pd
import pm4py
from pm4py.objects.petri_net.obj import PetriNet, Marking
from pm4py.objects.petri_net.utils import petri_utils
from pm4py.visualization.petri_net import visualizer as pn_visualizer

# Create a Petri net
net = PetriNet("petri_net_rules_1_to_8")

# Create transitions
t_fill_out_application = PetriNet.Transition("t_fill_out_application", "Fill out application")
t_architect_review1 = PetriNet.Transition("t_architect_review", "Architect Review")
t_lawyer_review1 = PetriNet.Transition("t_lawyer_review", "Lawyer Review")
t_architect_review2 = PetriNet.Transition("t_architect_review", "Architect Review")
t_lawyer_review2 = PetriNet.Transition("t_lawyer_review", "Lawyer Review")
t_reject = PetriNet.Transition("t_reject", "Reject")
t_applicant_informed = PetriNet.Transition("t_applicant_informed", "Applicant informed")
t_change_phase_to_abort = PetriNet.Transition("t_change_phase_to_abort", "Change phase to Abort")
t_change_phase_to_payout = PetriNet.Transition("t_change_phase_to_payout", "Change Phase to Payout")
t_first_payment = PetriNet.Transition("t_first_payment", "First payment")
t_undo_payment = PetriNet.Transition("t_undo_payment", "Undo payment")
t_change_phase_to_end_report1 = PetriNet.Transition("t_change_phase_to_end_report", "Change Phase to End Report")
t_change_phase_to_end_report2 = PetriNet.Transition("t_change_phase_to_end_report", "Change Phase to End Report")
t_account_number_changed = PetriNet.Transition("t_account_number_changed", "Account number changed")
t_approve_changed_account = PetriNet.Transition("t_approve_changed_account", "Approve changed account")


net.transitions.add(t_fill_out_application)
net.transitions.add(t_architect_review1)
net.transitions.add(t_lawyer_review1)
net.transitions.add(t_architect_review2)
net.transitions.add(t_lawyer_review2)
net.transitions.add(t_reject)
net.transitions.add(t_applicant_informed)
net.transitions.add(t_change_phase_to_abort)
net.transitions.add(t_change_phase_to_payout)
net.transitions.add(t_first_payment)
net.transitions.add(t_undo_payment)
net.transitions.add(t_change_phase_to_end_report1)
net.transitions.add(t_change_phase_to_end_report2)
net.transitions.add(t_account_number_changed)
net.transitions.add(t_approve_changed_account)


# Create places
p1 = PetriNet.Place("p1")
p2 = PetriNet.Place("p2")
p3 = PetriNet.Place("p3")
p4 = PetriNet.Place("p4")
p5 = PetriNet.Place("p5")
p6 = PetriNet.Place("p6")
p7 = PetriNet.Place("p7")
p8 = PetriNet.Place("p8")
p9 = PetriNet.Place("p9")
p10 = PetriNet.Place("p10")
p11 = PetriNet.Place("p11")
p12 = PetriNet.Place("p12")
p13 = PetriNet.Place("p13")

net.places.add(p1)
net.places.add(p2)
net.places.add(p3)
net.places.add(p4)
net.places.add(p5)
net.places.add(p6)
net.places.add(p7)
net.places.add(p8)
net.places.add(p9)
net.places.add(p10)
net.places.add(p11)
net.places.add(p12)
net.places.add(p13)

# Create arcs
# Rule 1
# The start state of the petrinet is initiated with a fill out application.
# This is the first transition because of rule 1 
petri_utils.add_arc_from_to(p1, t_fill_out_application, net)
petri_utils.add_arc_from_to(t_fill_out_application, p2,  net)

# Rule 2, 3
# The laywer and architect review happens directly after the fill out application
# Adhering to the rule 2. Additionally rule 3 is accomplished by making an OR 
# split that goes to either (laywer -> architect) or (architect -> laywer) but 
# NEVER at the same time. 

# (OR-SPLIT)
petri_utils.add_arc_from_to(p2, t_lawyer_review1, net)
petri_utils.add_arc_from_to(p2, t_architect_review1, net)

petri_utils.add_arc_from_to(t_lawyer_review1, p10, net)
petri_utils.add_arc_from_to(t_architect_review1, p3, net)

petri_utils.add_arc_from_to(p10, t_architect_review2, net)
petri_utils.add_arc_from_to(p3, t_lawyer_review2, net)


# (OR-JOIN)
petri_utils.add_arc_from_to(t_architect_review2, p4, net)
petri_utils.add_arc_from_to(t_lawyer_review2, p4, net)

# Rule 4, 7
# Rule 4 and rule 7 is initiated by making an OR-SPLIT. We will continue from
# rule 4
petri_utils.add_arc_from_to(p4, t_change_phase_to_payout, net)
petri_utils.add_arc_from_to(p4, t_reject, net)


# Rule 4, 8
# Rule 4's Reject should be followed by Applicant informed and Change phase
# to Abort is made with a AND-SPLIT as one transition is making an arch to two
# places.
# This is additionally the start of Rule 8 (part 1/2) with no restrictions to
# "Change phase to end report"
# (AND-SPLIT)
petri_utils.add_arc_from_to(t_reject, p5, net)
petri_utils.add_arc_from_to(t_reject, p6, net)

petri_utils.add_arc_from_to(p5, t_change_phase_to_abort, net)
petri_utils.add_arc_from_to(p6, t_applicant_informed, net)

petri_utils.add_arc_from_to(t_change_phase_to_abort, p7, net)
petri_utils.add_arc_from_to(t_applicant_informed, p8, net)


# (AND-JOIN)
petri_utils.add_arc_from_to(p7, t_change_phase_to_end_report1, net)
petri_utils.add_arc_from_to(p8, t_change_phase_to_end_report1, net)


# after the change phase to end report the flow can stop.
petri_utils.add_arc_from_to(t_change_phase_to_end_report1, p11, net)

# Rule 7,8
# when the change phase to payout is initiated, the first payment has to follow.
# Adhering to rule 7
# This is additionally the start of Rule 8 (part 2/2) with a restriction to 
# "Change phase to end report" not to happen before the "first payment" has taken place
petri_utils.add_arc_from_to(t_change_phase_to_payout, p9, net)
petri_utils.add_arc_from_to(p9, t_first_payment, net)

# Rule 6
# rule 6 if the account number has changed, then we approve the change and loop
# back to the place where the first payment can take place (no pun intended)
petri_utils.add_arc_from_to(p9, t_account_number_changed, net)
petri_utils.add_arc_from_to(t_account_number_changed, p13, net)
petri_utils.add_arc_from_to(p13, t_approve_changed_account, net)
petri_utils.add_arc_from_to(t_approve_changed_account, p9, net)

# Rule 2,3,5
# rule 5 When the first payment has been initiated, the flow can chose to undo
# the payment and loop back to the first payment (which is not the first anymore)
# and redo the process until satisfied. in which we go to Change change to end report

# NOTE that This "Change change to end report" is different (A copy) 
# from the OR-SPLIT from RULE 2 & 3. 
# This is necessary, because if the arc is hanging onto the origial 
# "Change to end report" then it would create an AND-JOIN, which means that the
# flow HAS to go through both a reject phase AND a payment phase in order to 
# continue. This is undesiable behavior.
petri_utils.add_arc_from_to(t_first_payment, p12, net)
petri_utils.add_arc_from_to(p12, t_undo_payment, net)
petri_utils.add_arc_from_to(t_undo_payment, p9, net)

petri_utils.add_arc_from_to(p12, t_change_phase_to_end_report2, net)
petri_utils.add_arc_from_to(t_change_phase_to_end_report2, p11, net)

initial_marking = Marking()
initial_marking[p1] = 1

final_marking = Marking()
final_marking[p11] = 1

# Visualize the Petri net
gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#Add rule 9 to the Petri net
ab_p1 = PetriNet.Transition("ab_p1", "Change phase to Abandon")
ab_p2 = PetriNet.Transition("ab_p2", "Change phase to Abandon")
ab_p3 = PetriNet.Transition("ab_p3", "Change phase to Abandon")
ab_p4 = PetriNet.Transition("ab_p4", "Change phase to Abandon")
ab_p5 = PetriNet.Transition("ab_p5", "Change phase to Abandon")
ab_p6 = PetriNet.Transition("ab_p6", "Change phase to Abandon")
ab_p7 = PetriNet.Transition("ab_p7", "Change phase to Abandon")
ab_p8 = PetriNet.Transition("ab_p8", "Change phase to Abandon")
ab_p9 = PetriNet.Transition("ab_p9", "Change phase to Abandon")
ab_p10 =PetriNet.Transition("ab_p10", "Change phase to Abandon")
ab_p11 =PetriNet.Transition("ab_p11", "Change phase to Abandon")
t_execute_abandon = PetriNet.Transition("t_execute_abandon", "Execute abandon")
#t_change_phase_to_abandon = PetriNet.Transition("t_change_phase_to_abandon", "Change phase to Abandon")
#print(net.transitions)
net.transitions.add(ab_p1)
net.transitions.add(ab_p2)
net.transitions.add(ab_p3)
net.transitions.add(ab_p4)
net.transitions.add(ab_p5)
net.transitions.add(ab_p6)
net.transitions.add(ab_p7)
net.transitions.add(ab_p8)
net.transitions.add(ab_p9)
net.transitions.add(ab_p10)
net.transitions.add(ab_p11)
net.transitions.add(t_execute_abandon)

petri_utils.add_arc_from_to(p2, ab_p1, net) 
petri_utils.add_arc_from_to(p3, ab_p2, net) 
petri_utils.add_arc_from_to(p4, ab_p3, net) 
petri_utils.add_arc_from_to(p5, ab_p4, net) 
petri_utils.add_arc_from_to(p6, ab_p5, net) 
petri_utils.add_arc_from_to(p7, ab_p6, net) 
petri_utils.add_arc_from_to(p8, ab_p7, net) 
petri_utils.add_arc_from_to(p9, ab_p8, net) 
petri_utils.add_arc_from_to(p10, ab_p9, net) 
petri_utils.add_arc_from_to(p12, ab_p10, net) 
petri_utils.add_arc_from_to(p13, ab_p11, net) 
interme = PetriNet.Place("interm")
net.places.add(interme)
petri_utils.add_arc_from_to(ab_p1, interme, net)
petri_utils.add_arc_from_to(ab_p2, interme, net)
petri_utils.add_arc_from_to(ab_p3, interme, net)
petri_utils.add_arc_from_to(ab_p4, interme, net)
petri_utils.add_arc_from_to(ab_p5, interme, net)
petri_utils.add_arc_from_to(ab_p6, interme, net)
petri_utils.add_arc_from_to(ab_p7, interme, net)
petri_utils.add_arc_from_to(ab_p8, interme, net)
petri_utils.add_arc_from_to(ab_p9, interme, net)
petri_utils.add_arc_from_to(ab_p10, interme, net)
petri_utils.add_arc_from_to(ab_p11, interme, net)


for i in range(0, 11):
    petri_utils.add_arc_from_to(interme, t_execute_abandon, net)

petri_utils.add_arc_from_to(t_execute_abandon, p11, net)

# Visualize the Petri net
gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)

## Task 2


For each rule describe how the rule is implemented (as a short paragraph for each rule).

The comments can be found in the code as well, which makes it easier to follow along

**Rule 1**
The start state of the petrinet is initiated with a fill out application. This is the first transition because of rule 1.

**Rule 2**
The laywer and architect review happens directly after the fill out application Adhering to the rule 2.

**Rule 3**
rule 3 is accomplished by making an OR split that goes to either (laywer -> architect) or (architect -> laywer) but NEVER at the same time. 

**Rule 4**
Rule 4 and rule 7 is initiated by making an OR-SPLIT. Rule 4's Reject should be followed by Applicant informed and Change phase to Abort is made with a AND-SPLIT as one transition is making an arch to two places.

**Rule 5**
Rule 5 When the first payment has been initiated, the flow can chose to undo the payment and loop back to the first payment (which is not the first anymore) and redo the process until satisfied. in which we go to Change change to end report

**Rule 6**
Rule 6 if the account number has changed, then we approve the change and loop back to the place where the first payment can take place (no pun intended)

**Rule 7**
Rule 4 and rule 7 is initiated by making an OR-SPLIT. We will continue from rule 7 when the change phase to payout is initiated, the first payment has to follow. Adhering to rule 7.

The OR-SPLIT from Rule 4 and Rule 7 The "Change change to end report" is different (A copy). It is not OR-JOINED from the OR-SPLIT from RULE 4 & 7. This is necessary, because if the arc is OR-JOINED to one and the origial "Change to end report" then it would create an AND-JOIN, which means that the flow HAS to go through both a reject phase AND a payment phase in order to continue. This is undesiable behavior.

**Rule 8**
Rule 8 (part 1/2) with no restrictions to "Change phase to end report".
Rule 8 (part 2/2) with a restriction to "Change phase to end report" not to happen before the "first payment" has taken place.

**Rule 9**
From any place (except the in marking) in the petri-net has been mapped to each individiaul transition "change phase to abandon". The reason that every place has their own transition, is because we want at any state (place) we wan to *chose* to change the phase to abandon. Whereafter a place that arcs to a transition "execute abandon" will end the petrinet.

If all the places were mapped to one transition, then it would function as an AND-JOIN, which would mean that EVERY place need to have a token before we can execute the abandonment.

## Task 3

Answer the following questions and add a short (1 paragraph) argument for your answer:

Petri-Net 1 is without rule 9.

Petri-Net 2 is with rule 9.

- **Is your Petri net live and/or quasi-live?** 

Petri-Net 1 is quasi-live from the initial marking, but not live. All transitions have the potential to fire from the first marking, but after the final marking has been reached, no transitions can fire. 

Petri-Net 2 is neither a quasi-live or live since we can achieve a deadlock. 
The deadlock can be achieved by firing the "reject" transition and then choose "change phase to abandon". "Change phase to end report" will no longer be able to fire and the deadlock has occured since we need the second token to close the AND-JOIN to the "Change phase to end report".


- **Is your Petri net bounded and/or safe?** 

Both Petri-Net 1 and Petri-Net 2 are safe and bounded. No place can have more than one token. Multiple tokens can be generated with AND-SPLITs by using OR-JOINS, unbounded tokens can then be created with loops.

We have an AND-SPLIT that produces two tokens in the connecting places, but this is followed by an AND-JOIN, which will join the tokens into one. If the process is abandoned, one token goes to the out place and the other token is stuck. 

We have two loops from "Change phase to payout" that can loop forever, but these loops does not include any AND-SPLITs and does therefore not create new tokens (i.e. if you have an AND-SPLIT that ends in an OR-JOIN). The same token can circulate but no more than one will be produced.


- **Is your Petri net a WorkFlow net?** 

Yes both Petri-Net 1 and Petri-Net 2 are WorkFlow nets. This in means that both have an in- and out-place and are strongly connected if we added an arc from out place (o) to in place (i). (i.e. there exists a path from any node to any other node. given the o to i definition).



# Part 2: Modelling Event Patterns as DCR Graphs

## Task 1

Model the rules from part 1 as individual DCR Graphs. (Yielding 9 graphs in total.)  Write a short description for each explaining the semantics. For convenience you may want to use an “other” event in some of your models that represents all events not explicitly mentioned.

In [ ]:
from pm4py.objects.dcr.obj import DcrGraph
graph = DcrGraph()

events = [
    "Fill out application", "Architect Review", "Lawyer Review", "Reject", 
    "Applicant informed", "Change phase to Abort", "Change Phase to Payout", 
    "First payment", "Undo payment", "Change Phase to End Report", 
    "Account number changed", "Approve changed account", "Execute abandon", 
    "Change phase to Abandon"
]

for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event

for event in graph.events:
    graph.marking.included.add(event)

pm4py.view_dcr(graph) 

## Task 2

Combine the separate patterns into a single model.

In [ ]:
print(dir(pm4py))

## Task 3

Answer the following questions:

**Which relations did you not use in your models?**
All the relations were used.

**Could some of the rules have been modelled in more than one way? If so, give one or two examples.**
We often discussed the A which has an enable relation to B where B is excluded to start with, is almost the same as A having a condition relation to B. For edge cases there is a semantic difference like 
in case one where B is excluded from the start. Another event (let's say C) can include this event and therefore nullifies the dependency on B. This can Case two not do as A is the sole dependency on B. 

**How does your model differ from the Petri net? Do they exhibit the same language?** The language is not the same in the two models. In the Petri-net we can only follow a specific sequence where we only do an activity once unless otherwise specified. In the DCR graph we can by default do an activity as many times as we like unless specified otherwise (using exclude). E.g. in the Petri-net we have to either the architect review and then the lawyer review OR the lawyer review and then the architect review. In the DCR graph, we can do as many reviews as we like in any order, we can just not do the two review-types at the same time.

